## Generate Sentences for Training CELLama from cellxgene census

In [1]:
import cellxgene_census
import scanpy
import tiledbsoma as soma

census_version = "2023-12-15"
organism = "homo_sapiens"

census = cellxgene_census.open_soma(census_version=census_version)

experiment = census["census_data"][organism]
query = experiment.axis_query(
    measurement_name="RNA",
    obs_query=soma.AxisQuery(),
)

In [2]:
query.n_obs


62998417

In [3]:
census_datasets = (
    census["census_info"]["datasets"]
    .read()#column_names=["collection_name", "dataset_title", "dataset_id", "soma_joinid"])
    .concat()
    .to_pandas()
)
census_datasets

,soma_joinid,collection_id,collection_name,collection_doi,dataset_id,dataset_version_id,dataset_title,dataset_h5ad_path,dataset_total_cell_count
0,0,4dca242c-d302-4dba-a68f-4c61e7bad553,Comparative transcriptomics reveals human-spec...,10.1126/science.ade9516,2bdd3a2c-2ff4-4314-adf3-8a06b797a33a,7eb7f2fd-fd74-4c99-863c-97836415652e,Human: Great apes study,2bdd3a2c-2ff4-4314-adf3-8a06b797a33a.h5ad,156285
1,1,d17249d2-0e6e-4500-abb8-e6c93fa1ac6f,Transcriptomic cytoarchitecture reveals princi...,10.1126/science.adf6812,f5b0810c-1664-4a62-ad06-be1d9964aa8b,d4427196-7876-4bdd-a929-ae4d177ec776,Dissection: Angular gyrus (AnG),f5b0810c-1664-4a62-ad06-be1d9964aa8b.h5ad,110752
2,2,d17249d2-0e6e-4500-abb8-e6c93fa1ac6f,Transcriptomic cytoarchitecture reveals princi...,10.1126/science.adf6812,e4ddac12-f48f-4455-8e8d-c2a48a683437,3280113b-7148-4a3e-98d4-015f443aab8a,Supercluster: CGE-derived interneurons,e4ddac12-f48f-4455-8e8d-c2a48a683437.h5ad,129495
3,3,d17249d2-0e6e-4500-abb8-e6c93fa1ac6f,Transcriptomic cytoarchitecture reveals princi...,10.1126/science.adf6812,e2808a6e-e2ea-41b9-b38c-4a08f1677f02,dc092185-3b8e-4fcb-ae21-1dc106d683ac,Dissection: Primary auditory cortex(A1),e2808a6e-e2ea-41b9-b38c-4a08f1677f02.h5ad,139054
4,4,d17249d2-0e6e-4500-abb8-e6c93fa1ac6f,Transcriptomic cytoarchitecture reveals princi...,10.1126/science.adf6812,d01c9dff-abd1-4825-bf30-2eb2ba74597e,c4959ded-83dc-4442-aac7-9a59bdb47801,Supercluster: Deep layer (non-IT) excitatory n...,d01c9dff-abd1-4825-bf30-2eb2ba74597e.h5ad,92969
...,...,...,...,...,...,...,...,...,...
646,646,180bff9c-c8a5-4539-b13b-ddbc00d643e6,Molecular characterization of selectively vuln...,10.1038/s41593-020-00764-7,f9ad5649-f372-43e1-a3a8-423383e5a8a2,0912e658-ccd7-43e9-8d81-4349432115f9,Molecular characterization of selectively vuln...,f9ad5649-f372-43e1-a3a8-423383e5a8a2.h5ad,8168
647,647,a72afd53-ab92-4511-88da-252fb0e26b9a,Single-cell atlas of peripheral immune respons...,10.1038/s41591-020-0944-y,456e8b9b-f872-488b-871d-94534090a865,fcc85817-ef31-4056-9a96-b3730ccec522,Single-cell atlas of peripheral immune respons...,456e8b9b-f872-488b-871d-94534090a865.h5ad,44721
648,648,38833785-fac5-48fd-944a-0f62a4c23ed1,Construction of a human cell landscape at sing...,10.1038/s41586-020-2157-4,2adb1f8a-a6b1-4909-8ee8-484814e2d4bf,998d8dbd-2f42-4611-9973-2da95db46c29,Construction of a human cell landscape at sing...,2adb1f8a-a6b1-4909-8ee8-484814e2d4bf.h5ad,598266
649,649,5d445965-6f1a-4b68-ba3a-b8f765155d3a,A molecular cell atlas of the human lung from ...,10.1038/s41586-020-2922-4,e04daea4-4412-45b5-989e-76a9be070a89,d54c8b65-3e6c-4265-98aa-5f082a5f3816,"Krasnow Lab Human Lung Cell Atlas, Smart-seq2",e04daea4-4412-45b5-989e-76a9be070a89.h5ad,9409


In [4]:
census["census_info"]["datasets"].read()

In [5]:
#Choose Dataset
dataset_id_query = census_datasets['dataset_id'].iloc[15]

In [6]:
hs_obs = (
    census["census_data"]["homo_sapiens"]
    .obs.read(value_filter=f"is_primary_data == True and dataset_id == '{dataset_id_query}'")
    .concat()
    .to_pandas()
)
hs_obs.shape

(216378, 26)

In [7]:
hs_obs.columns

Index(['soma_joinid', 'dataset_id', 'assay', 'assay_ontology_term_id',
       'cell_type', 'cell_type_ontology_term_id', 'development_stage',
       'development_stage_ontology_term_id', 'disease',
       'disease_ontology_term_id', 'donor_id', 'is_primary_data',
       'self_reported_ethnicity', 'self_reported_ethnicity_ontology_term_id',
       'sex', 'sex_ontology_term_id', 'suspension_type', 'tissue',
       'tissue_ontology_term_id', 'tissue_general',
       'tissue_general_ontology_term_id', 'raw_sum', 'nnz', 'raw_mean_nnz',
       'raw_variance_nnz', 'n_measured_vars'],
      dtype='object')

In [8]:
cell_subsampled_n = 1000
#cell_subsampled_ids = hs_obs["soma_joinid"].sample(cell_subsampled_n, random_state=1).tolist()
cell_subsampled_ids = hs_obs["soma_joinid"].tolist()

In [9]:
len(cell_subsampled_ids)

216378

In [10]:
adata_hs = cellxgene_census.get_anndata(
    census,
    organism="Homo sapiens",
    measurement_name="RNA",
    obs_coords=cell_subsampled_ids,
    var_coords=None,
)

In [11]:
adata_hs

AnnData object with n_obs × n_vars = 216378 × 60664
    obs: 'soma_joinid', 'dataset_id', 'assay', 'assay_ontology_term_id', 'cell_type', 'cell_type_ontology_term_id', 'development_stage', 'development_stage_ontology_term_id', 'disease', 'disease_ontology_term_id', 'donor_id', 'is_primary_data', 'self_reported_ethnicity', 'self_reported_ethnicity_ontology_term_id', 'sex', 'sex_ontology_term_id', 'suspension_type', 'tissue', 'tissue_ontology_term_id', 'tissue_general', 'tissue_general_ontology_term_id', 'raw_sum', 'nnz', 'raw_mean_nnz', 'raw_variance_nnz', 'n_measured_vars'
    var: 'soma_joinid', 'feature_id', 'feature_name', 'feature_length', 'nnz', 'n_measured_obs'

In [24]:
import sentence_from_adata_generator

In [25]:
from importlib import reload
reload(sentence_from_adata_generator)

<module 'sentence_from_adata_generator' from '/home/user/DATA1/Data/Xenium_lungcancer_10xdata/TrainingCELLama/sentence_from_adata_generator.py'>

In [26]:
num_samples = 1000  # Adjust based on the actual number of cells
top_k_values = [16, 20, 24]
obs_features_options = [None, ['tissue'], ['disease'], ['tissue_general'], ['assay']]
n_hvgs_s=[500,1500,3000]


In [ ]:
sentence_from_adata_generator.generate_and_save_examples(adata_hs, num_samples, top_k_values, obs_features_options, 
                                                         n_hvgs_s, save_file = f'./sentence_examples/temp_{dataset_id_query}.json')

Preprocessing data...


/home/user/anaconda3/envs/lang/lib/python3.9/site-packages/scanpy/preprocessing/_highly_variable_genes.py:226: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  disp_grouped = df.groupby("mean_bin")["dispersions"]


........ 500 16 None
----sample examples----
Texts: ['Top genes are CDH19, SCN7A, MT-CO1, ADGRB3, HIP1, MT-CYB, CLMN, NCAM2, EPB41L2, MT-ATP6, MT-ND1, MYH11, MT-ND4, IGFBP7, CASC15, and TIMP3.', 'Top genes are CDH19, SCN7A, MT-CO1, ADGRB3, NCAM2, MT-CO3, HES1, CLMN, AKT3, EPB41L2, MT-ND4, RHOB, MT-ATP6, COL5A3, TIMP3, and ELMO1.'], Label: 0.8480967879295349
Texts: ['Top genes are CDH19, KAZN, NEGR1, C7, TIMP3, PLA2G5, GSN, EBF1, ENG, ACSM3, SOX5, NCAM2, CRISPLD2, TCF4, AKT3, and RPL10.', 'Top genes are FHL2, TIMP3, RAD51B, ABCA6, TRAV40, TRAV38-2DV8, TRAV36DV7, TPT1, TMSB4X, TMSB10, TM4SF1, TLR9, TIMP1, THBS4, TRAV8-2, and TFPI.'], Label: -0.040422022342681885
Texts: ['Top genes are PLA2G5, MAML2, SPARCL1, NR2F2-AS1, TACC1, IGFBP5, AC007319.1, ADGRB3, GUCY1A2, CARMN, STEAP4, APBB2, ENPEP, RGS5-AS1, RGS5_ENSG00000143248, and SOX5.', 'Top genes are RPS18, RPL41, IGFBP7, RGS5_ENSG00000143248, RPS27, B2M, SPARCL1, RPS5, RPL23A, FABP4, RPL35, MT2A, TMSB4X, NDUFA4L2, ACTB, and RPS28.'], Labe